# Flight Price Predictions

This dataset contains information about flight prices and various features that might affect them. The data includes:

- Flight routes between different cities in India
- Various airlines
- Flight timings (departure and arrival)
- Duration of flights
- Additional information like number of stops
- Price of the flight tickets (target variable)

The goal is to analyze these features and build a prediction model for flight prices.


In [126]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

## Initial Data Loading

Import pandas library for data manipulation


In [127]:
df = pd.read_csv("./data/flight_price.csv")
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


## Load and Preview Dataset

Load the flight price dataset from CSV and display first few rows


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [129]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


## Feature Engineering of Dataset


In [130]:
df["Date"] = df["Date_of_Journey"].str.split("/").str[0]
df["Month"] = df["Date_of_Journey"].str.split("/").str[1]
df["Year"] = df["Date_of_Journey"].str.split("/").str[2]

## Date Feature Engineering

Split the Date_of_Journey column into separate Date, Month, and Year columns


In [131]:
df["Date"] = df["Date"].astype(int)
df["Month"] = df["Month"].astype(int)
df["Year"] = df["Year"].astype(int)

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
 11  Date             10683 non-null  int32 
 12  Month            10683 non-null  int32 
 13  Year             10683 non-null  int32 
dtypes: int32(3), int64(1), object(10)
memory usage: 1.0+ MB


In [133]:
df = df.drop("Date_of_Journey", axis=1)

In [134]:
df["Arrival_Time"] = df["Arrival_Time"].apply(lambda x: x.split(" ")[0])

## Arrival Time Processing

Clean and extract the arrival time information


In [135]:
df["Arrival_Hour"] = df["Arrival_Time"].str.split(":").str[0]
df["Arrival_Minute"] = df["Arrival_Time"].str.split(":").str[1]

df["Arrival_Hour"] = df["Arrival_Hour"].astype(int)
df["Arrival_Minute"] = df["Arrival_Minute"].astype(int)

In [136]:
df = df.drop("Arrival_Time", axis=1)

In [137]:
df["Dep_Hour"] = df["Dep_Time"].str.split(":").str[0]
df["Dep_Minute"] = df["Dep_Time"].str.split(":").str[1]

df["Dep_Hour"] = df["Dep_Hour"].astype(int)
df["Dep_Minute"] = df["Dep_Minute"].astype(int)

## Departure Time Processing

Extract hour and minute from departure time


In [138]:
df = df.drop("Dep_Time", axis=1)

In [139]:
df["Duration_in_Minutes"] = df["Duration"].str.extract(r"(\d+)h").fillna(0).astype(
    int
) * 60 + df["Duration"].str.extract(r"(\d+)m").fillna(0).astype(int)

## Duration Processing

Convert duration from string format (hours and minutes) to total minutes


In [140]:
df = df.drop("Duration", axis=1)

## Categorical Data Processing


In [141]:
df["Total_Stops"].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [142]:
df[df["Total_Stops"].isnull()]

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Duration_in_Minutes
9039,Air India,Delhi,Cochin,NaN,NaN,No info,7480,6,5,2019,9,25,9,45,1420


In [143]:
df["Total_Stops"] = df["Total_Stops"].map(
    {"1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4, "non-stop": 0, np.nan: 1}
)

#### Removing Route because i have alternative features.


In [144]:
df = df.drop("Route", axis=1)

In [ ]:
# one hot encoding
encoder = OneHotEncoder()

encoded_df = pd.DataFrame(
    encoder.fit_transform(df[["Airline", "Source", "Destination"]]).toarray(),
    columns=encoder.get_feature_names_out(),
)

In [ ]:
df = df.drop(["Airline", "Source", "Destination"], axis=1)

In [149]:
df = pd.concat([encoded_df, df], axis=1)

In [150]:
df.head()

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,...,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Duration_in_Minutes
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,No info,3897,24,3,2019,1,10,22,20,170
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,No info,7662,1,5,2019,13,15,5,50,445
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,No info,13882,9,6,2019,4,25,9,25,1140
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,No info,6218,12,5,2019,23,30,18,5,325
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,No info,13302,1,3,2019,21,35,16,50,285


In [151]:
df.describe()

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,...,Total_Stops,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Duration_in_Minutes
count,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,...,10683.000000,10683.000000,10683.000000,10683.000000,10683.0,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000
mean,0.029861,0.163999,0.018160,0.192174,0.360292,0.000562,0.111954,0.001217,0.076570,0.000094,...,0.824207,9087.064121,13.508378,4.708602,2019.0,13.348778,24.690630,12.490686,24.411214,643.093232
std,0.170210,0.370292,0.133535,0.394028,0.480108,0.023693,0.315324,0.034864,0.265921,0.009675,...,0.675199,4611.359167,8.479277,1.164357,0.0,6.859125,16.506036,5.748650,18.767980,507.862001
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1759.000000,1.000000,3.000000,2019.0,0.000000,0.000000,0.000000,0.000000,5.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,5277.000000,6.000000,3.000000,2019.0,8.000000,10.000000,8.000000,5.000000,170.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,8372.000000,12.000000,5.000000,2019.0,14.000000,25.000000,11.000000,25.000000,520.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,12373.000000,21.000000,6.000000,2019.0,19.000000,35.000000,18.000000,40.000000,930.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,4.000000,79512.000000,27.000000,6.000000,2019.0,23.000000,55.000000,23.000000,55.000000,2860.000000
